# MVP: Threat Modeling with AI (STRIDE)

This notebook builds an MVP that detects components in architecture diagrams and generates STRIDE-based threats and countermeasures.

## 1) Setup

In [ ]:
!apt-get -qq update && apt-get -qq install -y tesseract-ocr
!pip -q install ultralytics opencv-python pytesseract

In [ ]:
import os
from pathlib import Path
import cv2
import pytesseract
from ultralytics import YOLO

## 2) Dataset structure
Use YOLO format or COCO. Example for YOLO:
- data/images/train
- data/images/val
- data/labels/train
- data/labels/val

In [ ]:
DATA_DIR = Path('/content/data')
print('Data dir:', DATA_DIR)

## 3) Train detector (YOLOv8)

In [ ]:
# data.yaml should define train/val paths and class names
# Example classes: user, server, database, api, gateway, queue, storage, third_party, auth, frontend, backend, network
model = YOLO('yolov8n.pt')
# model.train(data='data.yaml', epochs=50, imgsz=640)

## 4) Inference

In [ ]:
# results = model.predict(source='sample.png', conf=0.25)
# detections = []
# for r in results:
#     for c in r.boxes.cls.tolist():
#         detections.append(model.names[int(c)])
# print(detections)

## 5) STRIDE mapping

In [ ]:
STRIDE_MAP = {
    'user': ['Spoofing', 'Elevation of Privilege'],
    'auth': ['Spoofing', 'Elevation of Privilege', 'Repudiation'],
    'api': ['Tampering', 'Repudiation', 'Denial of Service'],
    'gateway': ['Tampering', 'Repudiation', 'Denial of Service'],
    'database': ['Information Disclosure', 'Tampering'],
    'storage': ['Information Disclosure', 'Tampering'],
    'queue': ['Tampering', 'Denial of Service'],
    'frontend': ['Spoofing', 'Tampering', 'Information Disclosure'],
    'backend': ['Tampering', 'Repudiation', 'Denial of Service'],
    'network': ['Spoofing', 'Information Disclosure', 'Denial of Service'],
    'server': ['Tampering', 'Denial of Service', 'Elevation of Privilege'],
    'third_party': ['Spoofing', 'Information Disclosure', 'Repudiation']
}

def map_stride(components):
    out = {}
    for c in components:
        out[c] = STRIDE_MAP.get(c, [])
    return out

## 6) Report generation

In [ ]:
components = ['api', 'database', 'user']
stride = map_stride(components)

lines = ['# Threat Modeling Report', '']
for comp, threats in stride.items():
    lines.append(f'## {comp}')
    if threats:
        for t in threats:
            lines.append(f'- {t}')
    else:
        lines.append('- No threats mapped')
    lines.append('')

report = '
'.join(lines)
print(report)

with open('report.md', 'w', encoding='utf-8') as f:
    f.write(report)